In [1]:
# Set up IPython to show all outputs from a cell
import warnings
import numpy as np
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'



from numpy import load

idx_word = load('idx_word.npy').tolist()



from utils_2 import format_text, remove_spaces#, make_sequences, create_train_valid

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, load_model

def load_and_evaluate(model_dir1, model_name, return_model=False):
    """Load in a trained model and evaluate with log loss and accuracy"""

    model = load_model('{}/{}.h5'.format(model_dir1 ,model_name))
    #r = model.evaluate(X_valid, y_valid, batch_size=2048, verbose=1)

    #valid_crossentropy = r[0]
    #valid_accuracy = r[1]

    #print('Cross Entropy: {}'.format(round(valid_crossentropy, 4)))
    #print('Accuracy: {}%'.format(round(100 * valid_accuracy, 2)))

    if return_model:
        return model

In [3]:
#model_name = 'whitman'
model_dir = 'models'
import os
models = []
titles=[]
print("importing models: ")
for file in os.listdir(model_dir):
    if file.endswith(".h5"):
        titles.append(file[:-3])
        print(os.path.join(model_dir, file))
        model = load_and_evaluate(model_dir, file[:-3], return_model=True)
        models.append(model)
titles
len(models)



importing models: 
models/whitman.h5


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


models/frost.h5


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


models/dickinson.h5


...

/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


models/shakespeare.h5


/usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


['whitman', 'frost', 'dickinson', 'seuss', 'shakespeare']

5

In [4]:
from IPython.display import HTML
idx_word[1]

def stringToInts(input1):
    input2 = format_text(input1)
    array = input2.split(' ')
    #print(array)
    output = []
    for word in array:
        found = False
        for x, y in idx_word.items():
            if y == word:
                output.append(x)
                found=True
                break
        if not found:        
            print(str(word) + " not found in dict")
    return output
    
#stringToInts("walk to the. store yeet")


'\n'

In [5]:
import random


def generate_output(model,
                    sequences,
                    training_length=50,
                    new_words=100,
                    diversity=1,
                    return_output=False,
                    n_gen=1):
    """Generate `new_words` words of output from a trained model and format into HTML."""

   

    gen_list = []

    for n in range(n_gen):
        # Extract the seed sequence
        seed = stringToInts(sequences)
        #seed = [6,179,7,3,230,326,13,2]#,1589,1,247,1,3,109,134,16,1,2,3,73,681,1426,134,16,1,1590,4682,6,4683,13,2,396,6,1179,20,2845,1,6,84,67,2845,1,2,396,6,4684,49,59,1,3556]
        n2=[]
        for i in seed:
            n2.append(idx_word.get(i, '< --- >'))
        
        #seq[seed_idx:end_idx]
        #original_sequence = [idx_word[i] for i in seed]
        generated = seed[:] + ['#']

        # Find the actual entire sequence
        actual = generated[:]
        output = []
        # Keep adding new words
        for i in range(new_words):

            # Make a prediction from the seed
            preds = model.predict(np.array(seed).reshape(1, -1))[0].astype(
                np.float64)

            # Diversify
            preds = np.log(preds) / diversity
            exp_preds = np.exp(preds)

            # Softmax
            preds = exp_preds / sum(exp_preds)

            
            
            # Choose the next word
            probas = np.random.multinomial(1, preds, 1)[0]

            next_idx = np.argmax(probas)

            # New seed adds on old word
            seed = seed[1:] + [next_idx]
            generated.append(next_idx)
            output.append(next_idx)

        # Showing generated and actual abstract
        n = []

        for i in output:
            n.append(idx_word.get(i, '< --- >'))

        gen_list.append(n)
        #print(gen_list)
        
        

        #print("original")
        #print(remove_spaces(' '.join(n2)))
        #print("continuation:")
        #print(n)
        
    a = []

    gen_list = [
        gen[training_length:training_length + len(a)] for gen in gen_list
    ]

    if return_output:
        return original_sequence, gen_list, a

    # HTML formatting
    seed_html = ''
    seed_html = addContent(seed_html, header(
        'Seed Sequence', color='darkblue'))
    seed_html = addContent(seed_html,
                           box(remove_spaces(' '.join("original_sequence"))))

    gen_html = ''
    gen_html = addContent(gen_html, header('RNN Generated', color='darkred'))
    print(remove_spaces(' '.join(n)))
    gen_html = addContent(gen_html, box(remove_spaces(' '.join(n))))

    return seed_html, gen_html

In [6]:
def alliterate(word, prevWords):
    for otherword in prevWords:
        if word[0] == otherword[0]:
            return 1
    return 0

def rhymeswith(word, prevWords):
    return 0

def mesh_authors(model, author1, author2, 
                    sequences,
                    training_length=50,
                    new_words=100,
                    diversity=1,
                    return_output=False,
                    n_gen=1, auth1WEIGHT = .5):

    gen_list = []
    output_text = []
    print("generating output from " + titles[author1]+" and "+ titles[author2])
    for n in range(n_gen):
        # Extract the seed sequence
        seed = stringToInts(sequences)
        comprehended_words=[]
        for i in seed:
            comprehended_words.append(idx_word.get(i, '< --- >'))
        
        generated = seed[:] + ['#']
        # Find the actual entire sequence
        actual = generated[:]
        output = []
        # Keep adding new words
        for i in range(new_words):
            
            # predict next word for author 1
            pred1 = model[author1].predict(np.array(seed).reshape(1, -1))[0].astype(
                np.float64)
            # predict next word for author 2
            pred2 = model[author2].predict(np.array(seed).reshape(1, -1))[0].astype(
                np.float64)
            
                
            # Diversify
            pred1 = np.log(pred1) / diversity
            exp_preds = np.exp(pred1)
            # Softmax
            pred1 = exp_preds / sum(exp_preds)

            pred2 = np.log(pred2) / diversity
            exp_preds = np.exp(pred2)
            # Softmax
            pred2 = exp_preds / sum(exp_preds)
            
            
            
            #combine probability vectors
            pred1 = pred1 * auth1WEIGHT
            pred2 = pred2 * (1-auth1WEIGHT)
            
            preds = pred1 + pred2
            
            
            for pos in range(1,len(idx_word)):
                word = idx_word[pos]
                if rhymeswith(word, output_text[-40:]):
                    preds[pos] *=3;
                if alliterate(word, output_text[-2:]):
                    preds[pos] *=5;
            preds=preds/np.sum(preds)
            
            # Choose the next word
            probas = np.random.multinomial(1, preds, 1)[0]
            next_idx = np.argmax(probas)

            # New seed adds on old word
            seed = seed[1:] + [next_idx]
            generated.append(next_idx)
            output.append(next_idx)
            output_text.append(idx_word.get(next_idx, '< --- >'))
        # Showing generated and actual abstract
        
           
            
    return remove_spaces(' '.join(output_text))
idx_word[2]

','

In [7]:
for i in range(0,len(titles)):
    print(titles[i] + ": " + str(i))
    
print(mesh_authors(models, 0, , "test", new_words=50,diversity=1,auth1WEIGHT=1))

whitman: 0
frost: 1
dickinson: 2
seuss: 3
shakespeare: 4
generating output from whitman and dickinson
ballooning treasure tour defies d ga-zump dead-house launchest cities yellowstone cambodia snowy crevice tenons temple fulfill'd taste flatting themselves australia the any rest removed reverent artisans augmented snow-bank sluttish swerve clomping sarcastically sparkles weave dinner-table dissolute disappear'd ministers nativity stemming hegel several anchor selling sisters containing charging cheerfulest cherish'd juniper-tree


In [8]:
for i in range(0,len(titles)):
    print(titles[i] + ":")
    seed_html, gen_html = generate_output(models[i], "i ran away from you", 10,diversity=1)
    print("\n\n\n")
#HTML(seed_html)
#HTML(gen_html)

whitman:


NameError: name 'addContent' is not defined